In [ ]:
# Stocks Symbol List Data Source: https://datahub.io/core/nyse-other-listings#python
# Stock Data Source: FinnHub

In [1]:
# packages
# data
import finnhub
from datapackage import Package
import yfinance as yf

# scientific
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# utils
import time
import requests
import json



In [ ]:
################################################################################################

In [ ]:
# Gathering data from https://datahub.io/core/nyse-other-listings#python 
# -- on 01.12.2021 8497 symbols were captured
# stocks = []
# package = Package('https://datahub.io/core/nyse-other-listings/datapackage.json')

# # print list of all resources:
# # print(package.resource_names)

# # print processed tabular data (if exists any)
# for resource in package.resources:
#     if resource.descriptor['datahub']['type'] == 'derived/csv':
#         for entry in resource.read():
#             stocks.append(entry[0])

In [2]:
# EXTRA SYMBOLS
weed_stocks = ["GRWG", "JUSHF", "CRLBF", "HRVSF", "MJ", "CURLF", "CGC", "CRON", "MEDIF"]

tech_stocks = ["SNE", "XRX", "CSCO", "SIRI", "ROKU", "ATVI", "EA", "VZ", "T", "TMUS", "AMAT",
              "TXN", "MU", "FB", "MSFT", "SNAP", "AMZN", "TWTR", "AMD", "QCOM", "AVGO", "AAPL",
              "INTC", "NVDA", "NFLX"]

fin_stocks = ["AXP", "V", "MA", "JPM", "DFS", "MS", "BAC", "GS", "SQ", "PYPL", "GDOT"]

energy_stocks = ["NGG", "PCG", "RDS-B", "OXY", "MRO", "DD", "CVX", "DOW", "XOM", "PSX", "COP", 
              "HAL", "PLUG"]

travel_stocks = ["LUV", "MAR", "ABNB", "CCL", "RCL", "DIS", "AAL", "JBLU", "SAVE", "DAL", "HLT"]

food_consumer_stocks = ["DRI", "GIS", "NKE", "PG", "SHAK", "GRUB", "KDP", "PEP", "KO", "K", "QSR", "CMG",
              "YUM", "BGS", "ACI", "DAR", "KR", "GE", "GOOS", "HD", "LOW", "LULU", "DPZ", "SBUX", "MMM",
              "WMT", "TGT", "COST", "JACK", "MCD", "UBER", "GME"]

pharma_stocks = ["CVS", "RAD", "UNH", "LLY", "JNJ", "BMY", "AZN", "MRK", "ABBV", "GILD", "PFE"]

defense_infra_stocks = ["GD", "LMT", "NOC", "BA", "CAT", "WM"]

auto_stocks = ["HMC", "TM", "F", "TSLA", "GM"]

misc_stocks = ["ISR"]

all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
                + travel_stocks + food_consumer_stocks + pharma_stocks 
                + defense_infra_stocks + auto_stocks + misc_stocks)

# excluding pharma
# all_stocks = (weed_stocks + tech_stocks + fin_stocks + energy_stocks 
#               + travel_stocks + food_consumer_stocks + defense_infra_stocks 
#               + auto_stocks + misc_stocks)

# for symb in all_stocks:
#     if str(symb) not in stocks:
#         stocks.append(str(symb))

In [ ]:
# scrubbing data of symbols which contain bad_chars = ["$", ";", ":", "!", "*", "."]
# -- run until there is no more output
# bad_chars = ["$", ";", ":", "!", "*", "."]

# for s in stocks:
#     for char in bad_chars:
#         if char in str(s):
#             print(s)
#             stocks.remove(str(s))

In [3]:
display(len(all_stocks))

124

In [ ]:
################################################################################################

In [4]:
df = {}

In [5]:
# Adding company beta and 10_day_avg_volume values to a dataframe
count = 1
repeat = []

# Stock data
for stock in all_stocks:
# for stock in ["ABNB", ]:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        repeat.append(stock)
        time.sleep(30)
        pass
    
for stock in repeat:
    try:
        # request company's beta
        r = requests.get('https://finnhub.io/api/v1/stock/metric?symbol=' + str(stock) + '&metric=all&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()["metric"]
        count += 1

        beta = metrics["beta"]
        vol = metrics["10DayAverageTradingVolume"]
        if beta == None or vol == None or (str(stock) in df.keys()):
            continue

        # fill dictionary
        if beta >= 1.0:
            df[str(stock)] = {"beta": beta, "volume_10_day": vol}

    except:
        display(r.json())
        display(stock)
        time.sleep(30)
        pass
    

{'metric': {}, 'metricType': 'all', 'series': {}, 'symbol': 'MJ'}

'MJ'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'ABNB'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'TSLA'

{'error': 'API limit reached. Please try again later. Remaining Limit: 0'}

'GM'

{'metric': {}, 'metricType': 'all', 'series': {}, 'symbol': 'MJ'}

'MJ'

In [6]:
# dataframe to store stock data
data = pd.DataFrame(df)
data = data.transpose()
display(data)

,beta,volume_10_day
GRWG,3.12335,3.63656
CURLF,1.58976,1.58927
CGC,2.36008,1.68735
CRON,2.16141,1.19001
XRX,1.87889,2.05363
...,...,...
BA,1.61462,14.71303
F,1.23697,88.08976
ISR,1.51579,50.09557
TSLA,2.11570,43.80479


In [7]:
data['c'] = ''
data['o'] = ''
data['adx'] = ''
data['signal'] = ''
data['trending'] = ''
data['shareOutstanding'] = ''

In [8]:
# adding trends, signals, and price levels
count = 1
repeat = []

# Stock data
# for stock in ["MA", "CURLF", "AXP", "COP", "HAL", "DAR", "GOOS"]:
for stock in data.index:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        # trending metrics
        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]
        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]
        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares

        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        # add to dataframe
        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        repeat.append(stock)
        pass
    
print("repeating: " + str(repeat))
for stock in repeat:
    try:
        # Aggregate Indicators -- (signal, adx, trending)
        r = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol=' + str(stock) + '&resolution=W&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()
        count += 1

        signal = metrics["technicalAnalysis"]["signal"]
        adx = metrics["trend"]["adx"]
        trending = metrics["trend"]["trending"]

        # add to dataframe
        data.at[str(stock), 'adx'] = adx
        data.at[str(stock), 'signal'] = signal
        data.at[str(stock), 'trending'] = trending


        # company profile -- shareOutstanding 
        r = requests.get('https://finnhub.io/api/v1/stock/profile2?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        shares = r.json()["shareOutstanding"]

        # add to dataframe
        data.at[str(stock), 'shareOutstanding'] = shares


        # open price and current price
        r = requests.get('https://finnhub.io/api/v1/quote?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        # add to dataframe
        data.at[str(stock), 'o'] = metrics['o']
        data.at[str(stock), 'c'] = metrics['c']

        # News Sentiment
        r = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol=' + str(stock) + '&token=bvt5fmf48v6rku8bvmn0')
        metrics = r.json()

        data.at[str(stock), 'bearish'] = metrics["sentiment"]["bearishPercent"]
        data.at[str(stock), 'bullish'] = metrics["sentiment"]["bullishPercent"]

    except:
        print(str(count) + '...' + str(stock))
        time.sleep(30)
        pass

24...MS
25...BAC
41...MAR
57...GOOS
repeating: ['MS', 'BAC', 'MAR', 'GOOS']
73...GOOS


In [10]:
# data = data.sort_values(by=["signal", "beta", "shareOutstanding"], axis=0, ascending=False)
data = data.sort_values(by=["adx", "shareOutstanding"], axis=0, ascending=False)
# data.to_csv('./data/01_25_21_0800.csv', index=True)
# data = data.drop("RDS-B", axis=0)
n = len(data)
step = 15

data_1 = data[:step]
data_2 = data[step:step*2]
data_3 = data[step*2:step*3]
data_4 = data[step*3:]

ls = [data_1, data_2, data_3, data_4]

for df in ls:
    display(df)
#     break

,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
GOOS,1.55383,0.33756,,,,,,,NaN,NaN
GRWG,3.12335,3.63656,49.84,48.16,74.0775,buy,True,36.88,0.0000,1.0000
SNAP,1.29161,19.07045,53.57,53.9,62.9768,strong buy,True,1415.8,0.0000,1.0000
PLUG,1.71386,77.23934,66.87,62.3,59.6765,strong buy,True,303.38,0.2500,0.7500
DAR,1.28887,1.37371,66.78,66.69,58.7865,buy,True,163.78,0.5000,0.5000
GME,1.37064,34.07252,65.01,42.59,57.857,strong buy,True,64.3,0.5715,0.4285
ROKU,1.95563,5.58816,422.85,422,54.301,buy,True,119.9,0.0000,1.0000
SHAK,1.73000,1.08799,111.21,107.26,47.7416,strong buy,True,37.56,0.5000,0.5000
NVDA,1.40356,8.66204,548.5,549.604,46.8495,buy,True,613,0.0000,1.0000
AMD,2.19114,50.01342,92.79,94.42,43.1693,buy,True,1170,0.2500,0.7500


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
GE,1.08751,77.21625,11.11,10.97,40.9896,buy,True,8738.43,0.0000,1.0000
MU,1.26457,24.68311,82.28,84.08,40.1254,buy,True,1113,0.0000,1.0000
DFS,1.76691,2.78908,92.27,93.56,39.8027,buy,True,307,0.0000,1.0000
ISR,1.51579,50.09557,2.24,2.31,37.1227,buy,True,68.9,0.5000,0.5000
LUV,1.19432,6.89892,47.39,47.18,35.8518,buy,True,519.06,0.0000,1.0000
CURLF,1.58976,1.58927,15.07,14.98,35.6264,buy,True,470.09,0.5000,0.5000
CGC,2.36008,1.68735,33.8,33.15,35.4186,strong buy,True,350.11,0.5000,0.5000
PYPL,1.13454,6.82510,252,246.65,34.9013,strong buy,True,1173,0.0000,1.0000
AVGO,1.04776,1.64146,465.02,463.724,34.6058,buy,True,407,0.5000,0.5000
DIS,1.17671,8.31942,172.78,174,34.3875,buy,True,1781,0.1112,0.8888


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
BAC,1.55241,62.20462,31.55,31.36,32.2801,buy,True,8650.81,0.0000,1.0000
DD,1.67521,15.74439,81.09,80,31.387,buy,True,738.56,0.5000,0.5000
MS,1.51110,12.33978,74.13,72.55,30.2656,buy,True,0,0.0000,1.0000
GS,1.48588,4.10631,289.39,286.69,28.9284,buy,True,344.07,0.0000,1.0000
MRO,3.32941,35.31295,8.15,8.06,28.8094,neutral,True,796,0.5000,0.5000
JACK,1.73482,0.27103,101.71,98.86,28.5427,buy,True,22.72,0.0000,1.0000
CMG,1.31190,0.26181,1494.18,1492.32,28.0901,buy,True,27.75,0.0000,1.0000
OXY,2.45587,26.21254,21.49,20.67,27.1931,buy,True,894.11,0.5000,0.5000
JBLU,1.59281,8.63605,15.29,15.07,27.1617,buy,True,282,0.5000,0.5000
HLT,1.26807,2.28886,104.96,107.51,26.9315,buy,True,278.99,0.5000,0.5000


,beta,volume_10_day,c,o,adx,signal,trending,shareOutstanding,bearish,bullish
XOM,1.39414,34.57011,47.43,47.01,23.5374,buy,False,4234,0.8750,0.1250
AAL,1.84707,55.48026,15.82,15.57,23.3418,buy,False,428.2,0.5000,0.5000
COP,1.82339,12.96805,42.69,42.06,21.8342,buy,False,1084.87,0.5000,0.5000
PSX,1.70234,3.35282,72.55,71.7,21.671,buy,False,441.03,0.5000,0.5000
MAR,1.83804,2.20175,122.9,125.27,21.1471,buy,False,324,0.0000,1.0000
YUM,1.09614,1.50733,107.09,107.36,21.0394,buy,False,300,0.5000,0.5000
CVX,1.34117,11.28278,91.73,90.4,20.7806,buy,False,1882.17,0.5000,0.5000
TSLA,2.11570,43.80479,846.64,834.31,20.779,neutral,False,905,0.1715,0.8285
LOW,1.36445,4.08002,172.4,171.89,20.5342,neutral,False,763,0.0000,1.0000
BA,1.61462,14.71303,205.84,205.55,20.4244,buy,False,562.91,0.2500,0.7500


In [57]:
# for DEV
# pattern recognition
# for stock in data_1.index:
#     r = requests.get('https://finnhub.io/api/v1/scan/pattern?symbol=' + str(stock) + '&resolution=D&token=bvt5fmf48v6rku8bvmn0')
#     points = r.json()["points"]
#     for entry in points:
#         display(entry)

{'aprice': 1173.1700439453,
 'atime': 1604275200,
 'bprice': 1344.2700195312,
 'btime': 1604880000,
 'cprice': 1222.2199707031,
 'ctime': 1604966400,
 'dprice': 0,
 'dtime': 0,
 'end_price': 1344.2700195312,
 'end_time': 1607299200,
 'entry': 1344.27001953,
 'eprice': 0,
 'etime': 0,
 'mature': 0,
 'patternname': 'Double Bottom',
 'patterntype': 'bullish',
 'profit1': 1515.36999512,
 'profit2': 0,
 'sortTime': 1607299200,
 'start_price': 1344.2700195312,
 'start_time': 1603411200,
 'status': 'complete',
 'stoploss': 1156.06004639,
 'symbol': 'CMG.US',
 'terminal': 0}

{'aprice': 1182.9899902344,
 'atime': 1600646400,
 'bprice': 1287.8399658203,
 'btime': 1600819200,
 'cprice': 1214,
 'ctime': 1601596800,
 'dprice': 0,
 'dtime': 0,
 'end_price': 1287.8399658203,
 'end_time': 1601942400,
 'entry': 1287.83996582,
 'eprice': 0,
 'etime': 0,
 'mature': 0,
 'patternname': 'Double Bottom',
 'patterntype': 'bullish',
 'profit1': 1392.68994141,
 'profit2': 0,
 'sortTime': 1601942400,
 'start_price': 1287.8399658203,
 'start_time': 1600128000,
 'status': 'successful',
 'stoploss': 1172.50499268,
 'symbol': 'CMG.US',
 'terminal': 0}

{'aprice': 35.621402189347634,
 'atime': 1607904000,
 'bprice': 31.298770825808226,
 'btime': 1607904000,
 'cprice': 44.19122703796177,
 'ctime': 1608768000,
 'dprice': 39.868595674422366,
 'dtime': 1608768000,
 'entry': 44.191227037962,
 'entry_date': 1588896000,
 'eprice': 17.950000762939,
 'etime': 1604361600,
 'mature': 1,
 'patternname': 'Flag',
 'patterntype': 'bullish',
 'profit1': 48.513858401501,
 'profit2': 61.86262846437,
 'sortTime': 1608768000,
 'status': 'failed',
 'stoploss': 39.868595674422,
 'symbol': 'GRWG.US',
 'terminal': 0}

{'aprice': 37.189998626709,
 'atime': 1606694400,
 'bprice': 32.229999542236,
 'btime': 1606780800,
 'cprice': 36.5,
 'ctime': 1607040000,
 'dprice': 0,
 'dtime': 0,
 'end_price': 32.229999542236,
 'end_time': 1607385600,
 'entry': 32.22999954,
 'eprice': 0,
 'etime': 0,
 'mature': 0,
 'patternname': 'Double Top',
 'patterntype': 'bearish',
 'profit1': 27.27000046,
 'profit2': 0,
 'sortTime': 1607385600,
 'start_price': 32.229999542236,
 'start_time': 1606262400,
 'status': 'failed',
 'stoploss': 37.68599854,
 'symbol': 'GRWG.US',
 'terminal': 0}

{'aprice': 17.889999389647997,
 'atime': 1597968000,
 'bprice': 14.45211327502134,
 'btime': 1597968000,
 'cprice': 15.201735526313197,
 'ctime': 1598918400,
 'dprice': 11.76384941168654,
 'dtime': 1598918400,
 'entry': 15.201735526313,
 'entry_date': 1599004800,
 'eprice': 6.8899998664856,
 'etime': 1594684800,
 'mature': 1,
 'patternname': 'Flag',
 'patterntype': 'bullish',
 'profit1': 18.63962164094,
 'profit2': 26.201735049476,
 'sortTime': 1598918400,
 'status': 'success',
 'stoploss': 11.763849411687,
 'symbol': 'GRWG.US',
 'terminal': 0}

...

{'aprice': 52.169998168945,
 'atime': 1593129600,
 'bprice': 56.740001678467,
 'btime': 1593993600,
 'cprice': 53.619998931885,
 'ctime': 1594684800,
 'dprice': 0,
 'dtime': 0,
 'end_price': 56.740001678467,
 'end_time': 1594771200,
 'entry': 56.74000168,
 'eprice': 0,
 'etime': 0,
 'mature': 0,
 'patternname': 'Double Bottom',
 'patterntype': 'bullish',
 'profit1': 61.31000519,
 'profit2': 0,
 'sortTime': 1594771200,
 'start_price': 56.740001678467,
 'start_time': 1592524800,
 'status': 'failed',
 'stoploss': 51.71299782,
 'symbol': 'QSR.US',
 'terminal': 0}

{'aprice': 110.55999755859,
 'atime': 1609200000,
 'dprice': 105.23999786377,
 'dtime': 1609804800,
 'mature': 1,
 'patternname': 'two black gapping',
 'patterntype': 'bearish',
 'sortTime': 1609804800,
 'status': 'complete',
 'symbol': 'YUM.US'}

{'aprice': 107.69999694824,
 'atime': 1606435200,
 'dprice': 104.80000305176,
 'dtime': 1606867200,
 'mature': 1,
 'patternname': 'two black gapping',
 'patterntype': 'bearish',
 'sortTime': 1606867200,
 'status': 'complete',
 'symbol': 'YUM.US'}

{'aprice': 98.650001525879,
 'atime': 1599004800,
 'dprice': 93.480003356934,
 'dtime': 1599523200,
 'mature': 1,
 'patternname': 'three black crows',
 'patterntype': 'bearish',
 'sortTime': 1599523200,
 'status': 'complete',
 'symbol': 'YUM.US'}

{'aprice': 42.849998474121,
 'atime': 1588896000,
 'dprice': 35.75,
 'dtime': 1589328000,
 'mature': 1,
 'patternname': 'three black crows',
 'patterntype': 'bearish',
 'sortTime': 1589328000,
 'status': 'complete',
 'symbol': 'DFS.US'}